# 2048 Command Line Interface

Matheus Schmitz<br>
<a href="https://www.linkedin.com/in/matheusschmitz/">LinkedIn</a><br>
<a href="https://matheus-schmitz.github.io/">Github Portfolio</a>

## Imports

In [1]:
import random
import copy

## Game

In [2]:
BOARD_SIZE = 4

In [3]:
# Print the current board
def display():
    # Find the largest value currently on the board
    largest = board[0][0]
    for row in board:
        for element in row:
            if element > largest:
                largest = element

    # Set the number of spaces needed to the length of the lastest value
    numSpaces = len(str(largest))
    
    for row in board:
        currRow = '|'
        for element in row:
            if element == 0:
                currRow += ' ' * numSpaces + '|'
            else:
                currRow += (' ' * (numSpaces - len(str(element)))) + str(element) + '|'
        print(currRow)
    print()

In [4]:
# Merge one row left
def mergeOneRowL(row):
    # Move evertyhing as far to the left as possible
    for j in range(BOARD_SIZE -1):
        for i in range(BOARD_SIZE -1, 0, -1):
            # Test if there is an empty space, move over if so
            if row[i - 1] == 0:
                row[i - 1] = row[i]
                row[i] = 0
    
    # Merge everything to the left
    for i in range(BOARD_SIZE - 1):
        # Test if the current value is identical to the one next to it
        if row[i] == row[i + 1]:
            row[i] *= 2
            row[i + 1] = 0
    
    # Move everything to the left again
    for i in range(BOARD_SIZE -1, 0, -1):
        # Test if there is an empty space, move over if so
        if row[i - 1] == 0:
            row[i - 1] = row[i]
            row[i] = 0
    
    return row

In [5]:
# Merge the whole board to the left
def merge_left(currentBoard):
    # Merge every row in the board left
    for i in range(BOARD_SIZE):
        currentBoard[i] = mergeOneRowL(currentBoard[i])
    return currentBoard

In [6]:
# Reverse the order of a row
def reverse(row):
    # Add all elements of the row to a new list, in reverse order
    new = []
    for i in range(BOARD_SIZE -1, -1, -1):
        new.append(row[i])
    return new

In [7]:
# Merge the whole board to the right
def merge_right(currentBoard):
    # Look at every row in the board
    for i in range(BOARD_SIZE):
        # Reverse the row, merge to the left, then reverse back
        currentBoard[i] = reverse(currentBoard[i])
        currentBoard[i] = mergeOneRowL(currentBoard[i])
        currentBoard[i] = reverse(currentBoard[i])

In [8]:
# Transpose the whole board
def transpose(currentBoard):
    for j in range(BOARD_SIZE):
        for i in range(j, BOARD_SIZE):
            currentBoard[i][j], currentBoard[j][i] = currentBoard[j][i], currentBoard[i][j]
    return currentBoard

In [9]:
# Merge the whole board up
def merge_up(currentBoard):
    # Transposes the whole board, merge it all left, then transposes it back
    currentBoard = transpose(currentBoard)
    currentBoard = merge_left(currentBoard)
    currentBoard = transpose(currentBoard)
    return currentBoard

In [10]:
# Merge the whole board down
def merge_down(currentBoard):
    # Transposes the whole board, merge it all right, then transposes it back
    currentBoard = transpose(currentBoard)
    currentBoard = merge_right(currentBoard)
    currentBoard = transpose(currentBoard)
    return currentBoard

In [11]:
# Pick a new value to add to the board
def pickNewValue():
    if random.randint(1, 8) == 1:
        return 4
    else:
        return 2

In [12]:
# Add a value to the board in one of the empty spaces
def addNewValue():
    rowNum = random.randint(0, BOARD_SIZE - 1)
    colNum = random.randint(0, BOARD_SIZE - 1)

    # Pick spots until we find one that is empty
    while not board[rowNum][colNum] == 0:
        rowNum = random.randint(0, BOARD_SIZE - 1)
        colNum = random.randint(0, BOARD_SIZE - 1)

    # Fill the empty spot with a new value
    board[rowNum][colNum] = pickNewValue()

In [18]:
# Check if the player won
def won():
    if any(2048 in row for row in board):
        return Truen
    return False

In [19]:
# Check if the player lost
def noMoves():
    # Create two copies of the board
    tempBoard1 = copy.deepcopy(board)
    tempBoard2 = copy.deepcopy(board)
    
    hasValidMove = False
    
    # Test every possible move
    tempBoard1 = merge_up(tempBoard1)
    if tempBoard1 != tempBoard2:
        hasValidMove = True
        
    tempBoard1 = merge_down(tempBoard1)
    if tempBoard1 != tempBoard2:
        hasValidMove = True
        
    tempBoard1 = merge_left(tempBoard1)
    if tempBoard1 != tempBoard2:
        hasValidMove = True
        
    tempBoard1 = merge_right(tempBoard1)
    if tempBoard1 != tempBoard2:
        hasValidMove = True
        
    return not(hasValidMove)

In [20]:
# Create a blank board
board = [[0] * BOARD_SIZE for _ in range(BOARD_SIZE)]

In [21]:
# Fill two spots with random values, to start the game
numNeeded = 2
while numNeeded > 0:
    rowNum = random.randint(0, BOARD_SIZE - 1)
    colNum = random.randint(0, BOARD_SIZE - 1)
    
    if board[rowNum][colNum] == 0:
        board[rowNum][colNum] = pickNewValue()
        numNeeded -= 1

In [22]:
print("Welcome to 2048! Your goal is to combine values to get the number 2048, by mergerd the board in different directions. Everytime, you will need to type 'd' to merge right, 'w' to merge up, 'a' to merge left, and 's' to merge down.")

display()

gameOver = False

# Repeat asking the user for new moves while the game isn't over
while not gameOver:
    move = input("Which way do you want to merge?")
    
    # Assume they entered a valid input
    validInput = True
    
    # Create a copy of the board
    tempBoard = copy.deepcopy(board)
    
    # Figure out which way the person wants to merge and use the correct function
    if move == 'w':
        board = merge_up(board)
    elif move == 'a':
        board = merge_left(board)
    elif move == 's':
        board = merge_down(board)
    elif move == 'd':
        board = merge_right(board)
    else:
        validInput = False
    
    # If the input was not valid, request the user to enter a valid input
    if not validInput:
        print("Your input was not valid, please try again")
    # Otherwise the input was valid
    else:
        # Test if the user has won
        if won():
            display()
            print("You Won!")
            gameOver = True
        else:
            # Test if their move was unsuccessful
            if board == tempBoard:
                print("Try a different direction")
            else:
                addNewValue()
                display()
                if noMoves:
                    print("Sorry, you have no more possible moves, you lose!")
                    gameOver = True

Welcome to 2048! Your goal is to combine values to get the number 2048, by mergerd the board in different directions. Everytime, you will need to type 'd' to merge right, 'w' to merge up, 'a' to merge left, and 's' to merge down.
| | | | |
|2| | | |
| |2| | |
| | | | |

Which way do you want to merge?
Your input was not valid, please try again
Which way do you want to merge?w
|2|2| | |
| | | | |
| | | | |
| | | |2|

Sorry, you have no more possible moves, you lose!
